# Example 6.2

In [1]:
import pathlib
import numpy as np
import pandas as pd
import numpy.linalg as la
from IPython.display import display, Math
from scipy import stats

Table 6.2 contains the four measurements for each of the 19 dogs, where

$$
    \begin{align*}
        \text{Treatment 1} &= \text{high CO2 pressure without H} \\
        \text{Treatment 2} &= \text{low CO2 pressure without H} \\
        \text{Treatment 3} &= \text{high CO2 pressure with H} \\
        \text{Treatment 4} &= \text{low CO2 pressure with H}
    \end{align*}
$$

In [2]:
def load_data() -> pd.DataFrame:
    # Load the sleeping dog data.
    fpth = pathlib.WindowsPath(r'..\..\data\Table6.2.xlsx')
    df = pd.read_excel(fpth).drop(columns='Dog')
    return df

In [3]:
dog_df = load_data()
dog_df.head(3)

,Treatment1,Treatment2,Treatment3,Treatment4
0,426,609,556,600
1,253,236,392,395
2,359,433,349,357


$$
    \begin{align*}
    (\mu_{3} + \mu_{4}) - (\mu_{1} + \mu_{2}) &= \left(\begin{matrix} \text{Halothane contrast representing the} \\ \text{difference between the presence and} \\ \text{absence of halothane}\end{matrix}\right) \\
    (\mu_{1} + \mu_{3}) - (\mu_{2} + \mu_{4}) &= \left(\begin{matrix} \text{CO}_{2}\text{ contrast representing the difference} \\ \text{between high and low }\text{CO}_{2}\text{ pressure}\end{matrix}\right) \\
    (\mu_{1} + \mu_{4}) - (\mu_{2} + \mu_{3}) &= \left(\begin{matrix} \text{Contrast representing the influence} \\ \text{of halothane on }\text{CO}_{2}\text{ pressure differences} \\ \text{(H-CO}_{2}\text{ pressure "interaction")}\end{matrix}\right)
    \end{align*}
$$

In [4]:
X = dog_df.to_numpy()
n, p = X.shape
C = np.array([[-1, -1,  1,  1],
              [ 1, -1,  1, -1],
              [ 1, -1, -1,  1]]
             )
q = C.shape[1]

In [5]:
xbar = np.mean(X, axis=0)[:, np.newaxis]
S = np.cov(X, rowvar=False)

In [6]:
Cxbar = C @ xbar
CSC = C @ S @ C.T

Creating an $\alpha=0.05$-level test of the hypothesis
$$
    \begin{align*}
        H_{0}: \textbf{C}\bm{\mu} = &\textbf{0} \\
        H_{a}: \textbf{C}\bm{\mu} \ne &\textbf{0}
    \end{align*}
$$

$$
    T^{2}
    =
    n
    {\left(\textbf{C}\bar{\textbf{x}}\right)}^{\prime}
    {\left(\textbf{C}\textbf{S}\textbf{C}\right)}^{-1}
    \left(\textbf{C}\bar{\textbf{x}}\right)
$$

In [7]:
T2 = n*(Cxbar.T @ la.inv(CSC) @ Cxbar).item()
T2

116.01632120098769

$$
    \frac{(n-1)(q-1)}{n-(q-1)}
    F_{q-1, n-(q-1)}
    (\alpha)
$$

In [8]:
const = ((n-1)*(q-1))/(n-(q-1))
f_val = stats.f.ppf(1-0.05, dfn=q-1, dfd=n-(q-1))
f_crit = const*f_val
f_crit

10.931191371405848

Print out the result

In [9]:
if T2 <= f_crit:
    display(Math(fr'\text{{Fail to reject }}H_{{0}}: C\bar{{\textbf{{x}}}} = \textbf{{0}}'
                fr'\text{{, since }}T^{{2}} = {T2:.3f} \le '
                fr'\frac{{(n-1)(q-1)}}{{n-(q-1)}}F_{{q-1, n-(q-1)}}(0.05) = {f_crit:.3f}'))
else:
    display(Math(fr'\text{{Reject }}H_{{0}}: C\bar{{\textbf{{x}}}} = \textbf{{0}}'
                 fr'\text{{ in favor of }}H_{{a}}: \textbf{{C}}\bar{{\textbf{{x}}}} \ne \textbf{{0}}'
                fr'\text{{, since }}T^{{2}} = {T2:.3f} > '
                fr'\frac{{(n-1)(q-1)}}{{n-(q-1)}}F_{{q-1, n-(q-1)}}(0.05) = {f_crit:.3f}'))

<IPython.core.display.Math object>

Create 95% simultaneous confidence intervals

In [10]:
ci95 = Cxbar + np.array([-1, 1]) * np.sqrt(f_crit) * np.sqrt(np.diag(CSC)[:, np.newaxis] / n)
ci95

array([[ 135.65029727,  282.98128168],
       [-114.72708272,   -5.37818044],
       [ -78.72858439,   53.14963702]])

In [11]:
# Text descriptions for each contrast.
desc_text = [r'\text{halothane influence}',
             r'CO_{2}\text{ pressure influence }',
             r'\text{H-CO}_{2}\text{ pressure "interaction" }']

In [12]:
for i in range(ci95.shape[0]):
    display(Math(desc_text[i] + fr' = {Cxbar[i].item():.3f} \pm '
                fr'\sqrt{{{f_crit:.3f}}} '
                fr'\sqrt{{\frac{{{np.diag(CSC)[i]:.3f}}}{{{n}}}}} = '
                fr'{Cxbar[i].item():.3f} \pm {np.sqrt(f_crit*(np.diag(CSC)[i]/n)):.3f} '
                fr'\hspace{{0.4cm}}\text{{ or as }}\hspace{{0.4cm}}'
                fr'({ci95[i,0]:.3f}, {ci95[i,1]:.3f})'
                ))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>